In [4]:
pwd

'D:\\ML\\EVA\\JEDI\\Captsone_EVA5'

In [4]:
os.sep

'\\'

In [3]:
from model import *
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = fork(depth_freeze = False, yolo_freeze = True,depth_preload_pth = 'D:/ML/EVA/JEDi/Midas/model-f46da743.pt', yolo_preload_pth = 'D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt').to(device)
from torchsummary import summary
print(summary(model, (3,416,416)))

Using cache found in C:\Users\realp/.cache\torch\hub\facebookresearch_WSL-Images_master


encoder loaded weights from D:/ML/EVA/JEDi/Midas/model-f46da743.pt
resnet_layer 0 was frozen
resnet_layer 1 was frozen
resnet_layer 2 was frozen
resnet_layer 3 was frozen
depth_decoder loaded from D:/ML/EVA/JEDi/Midas/model-f46da743.pt
loaded weight from D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt
yolo_decoder loaded from D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt
yolo_layer 0 was frozen
yolo_layer 1 was frozen
yolo_layer 2 was frozen
yolo_layer 3 was frozen
yolo_layer 4 was frozen
yolo_layer 5 was frozen
yolo_layer 6 was frozen
yolo_layer 7 was frozen
yolo_layer 8 was frozen
yolo_layer 9 was frozen
yolo_layer 10 was frozen
yolo_layer 11 was frozen
yolo_layer 12 was frozen
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 208, 208]           9,408
       BatchNorm2d-2         [-1, 64, 208, 208]             128
              ReLU-3         [-1, 64, 208, 208]               0
    

# Train.py for yolo-WIP

In [29]:
cd D:\ML\EVA\YoloV3

D:\ML\EVA\YoloV3


In [2]:
!python fork_train.py --data D:/ML/EVA/YoloV3/data/customdata/custom.data --batch 10 --cache --epochs 10 --nosave

Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 

2020-11-19 19:52:29.480287: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
Using cache found in C:\Users\realp/.cache\torch\hub\facebookresearch_WSL-Images_master

Caching labels:   0%|          | 0/6 [00:00<?, ?it/s]
Caching labels (6 found, 0 missing, 0 empty, 0 duplicate, for 6 images): 100%|##########| 6/6 [00:00<00:00, 6210.72it/s]

Caching images (0.0GB): 100%|##########| 6/6 [00:00<00:00, 36.68it/s]

Caching labels:   0%|          | 0/6 [00:00<?, ?it/s]
Caching labels (6 found, 0 missing, 0 empty, 0 duplicate, for 6 images): 100%|##########| 6/6 [00:00<00:00, 6016.21it/s]

Caching images (0.0GB): 100%|##########| 6/6 [00:00<00:00, 55.20it/s]

  0%|          | 0/1 [00:00<?, ?it/s]
       0/9      3.3G      2.92      99.2      2.76       105        27       512:   0%|          | 0/1 [00:02<?, ?it/s]D:\ML\EVA\JEDI\Captsone_EVA5\utils_yolo\utils.py:880: MatplotlibDeprecationWarning: Passing non-integers as three-e

Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 


               Class    Images   Targets         P         R   mAP@0.5        F1: 100%|##########| 1/1 [00:12<00:00, 12.40s/it]
               Class    Images   Targets         P         R   mAP@0.5        F1: 100%|##########| 1/1 [00:12<00:00, 12.70s/it]

  0%|          | 0/1 [00:00<?, ?it/s]
       5/9     4.12G      2.68      41.6      3.01      47.3        26       512:   0%|          | 0/1 [00:00<?, ?it/s]
       5/9     4.12G      2.68      41.6      3.01      47.3        26       512: 100%|##########| 1/1 [00:00<00:00,  1.06it/s]
       5/9     4.12G      2.68      41.6      3.01      47.3        26       512: 100%|##########| 1/1 [00:01<00:00,  1.24s/it]

               Class    Images   Targets         P         R   mAP@0.5        F1:   0%|          | 0/1 [00:00<?, ?it/s]
               Class    Images   Targets         P         R   mAP@0.5        F1: 100%|##########| 1/1 [00:12<00:00, 12.42s/it]
               Class    Images   Targets         P         R   mAP@0.5        F


Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Model Summary: 225 layers, 6.25895e+07 parameters,

In [2]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import os
from model import *


Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients


In [3]:
wdir = 'weights' + os.sep  # weights dir
last = wdir + 'last.pt'
best = wdir + 'best.pt'
results_file = 'results.txt'

hyp = {'giou': 3.54,  # giou loss gain
       'cls': 37.4,  # cls loss gain
       'cls_pw': 1.0,  # cls BCELoss positive_weight
       'obj': 64.3,  # obj loss gain (*=img_size/320 if img_size != 320)
       'obj_pw': 1.0,  # obj BCELoss positive_weight
       'iou_t': 0.225,  # iou training threshold
       'lr0': 0.01,  # initial learning rate (SGD=5E-3, Adam=5E-4)
       'lrf': 0.0005,  # final learning rate (with cos scheduler)
       'momentum': 0.937,  # SGD momentum
       'weight_decay': 0.000484,  # optimizer weight decay
       'fl_gamma': 0.0,  # focal loss gamma (efficientDet default is gamma=1.5)
       'hsv_h': 0.0138,  # image HSV-Hue augmentation (fraction)
       'hsv_s': 0.678,  # image HSV-Saturation augmentation (fraction)
       'hsv_v': 0.36,  # image HSV-Value augmentation (fraction)
       'degrees': 1.98 * 0,  # image rotation (+/- deg)
       'translate': 0.05 * 0,  # image translation (+/- fraction)
       'scale': 0.05 * 0,  # image scale (+/- gain)
       'shear': 0.641 * 0}  # image shear (+/- deg)

from types import SimpleNamespace   
d = dict(data='D:/ML/EVA/YoloV3/data/customdata/custom.data',
epochs = 2, 
batch_size = 10, 
multi_scale = False,
cache_images = True,
img_size = [512, 512, 512], 
accumulate = 4,
single_cls = False,
adam = False,
rect = False,
notest = False,
nosave = True,
name = ''         
        )
opt = SimpleNamespace(**d)

# train 
data = opt.data
epochs = opt.epochs
batch_size = opt.batch_size
accumulate = opt.accumulate  # effective bs = batch_size * accumulate = 16 * 4 = 64
#weights = opt.weights  # initial training weights
imgsz_min, imgsz_max, imgsz_test = opt.img_size

#
init_seeds()

data_dict = parse_data_cfg(data)
train_path = data_dict['train']
test_path = data_dict['valid']
nc = 1 if opt.single_cls else int(data_dict['classes'])  # number of classes
hyp['cls'] *= nc / 80  # update coco-tuned hyp['cls'] to current dataset
for f in glob.glob('*_batch*.png') + glob.glob(results_file):
    os.remove(f)

In [3]:
'D:/ML/EVA/YoloV3/' + data_dict['train']

'D:/ML/EVA/YoloV3/data/customdata/train.txt'

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = fork(depth_freeze = True, yolo_freeze = False,depth_preload_pth = '', yolo_preload_pth = 'D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt').to(device)

Using cache found in C:\Users\realp/.cache\torch\hub\facebookresearch_WSL-Images_master


encoder loaded weights from D:/ML/EVA/JEDI/MiDaS/model-f46da743.pt
resnet_layer 0 was frozen
resnet_layer 1 was frozen
resnet_layer 2 was frozen
resnet_layer 3 was frozen
depth_layer 0 was frozen
loaded weight from D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt
yolo_decoder loaded from D:/ML/EVA/JEDI/YoloV3master/last_ppe.pt


In [5]:
pg0, pg1, pg2 = [], [], []  # optimizer parameter groups
for k, v in dict(model.named_parameters()).items():
    if v.requires_grad: 
        if '.bias' in k:
            pg2 += [v]  # biases
        elif '.weight' in k and 'Conv2d' in k:
            pg1 += [v]  # apply weight_decay
        else:
            pg0 += [v]  # all else
#len(pg2) + len(pg1) + len(pg0), len(pg2),len(pg1),len(pg0)

if opt.adam:
    # hyp['lr0'] *= 0.1  # reduce lr (i.e. SGD=5E-3, Adam=5E-4)
    optimizer = optim.Adam(pg0, lr=hyp['lr0'])
    # optimizer = AdaBound(pg0, lr=hyp['lr0'], final_lr=0.1)
else:
    optimizer = optim.SGD(pg0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)
optimizer.add_param_group({'params': pg1, 'weight_decay': hyp['weight_decay']})  # add pg1 with weight_decay
optimizer.add_param_group({'params': pg2})  # add pg2 (biases)
del pg0, pg1, pg2

#mixed precision
start_epoch = 0
best_fitness = 0.0
mixed_precision = True
try:  # Mixed precision training https://github.com/NVIDIA/apex
    from apex import amp
except:
    # print('Apex recommended for mixed precision and faster training: https://github.com/NVIDIA/apex')
    mixed_precision = False
if mixed_precision:
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1', verbosity=0)

#scheduler
lf = lambda x: (((1 + math.cos(
    x * math.pi / epochs)) / 2) ** 1.0) * 0.95 + 0.05  # cosine https://arxiv.org/pdf/1812.01187.pdf
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf, last_epoch=start_epoch - 1)

In [6]:
from utils_yolo.datasets import * 

In [7]:
cd D:\ML\EVA\YoloV3

D:\ML\EVA\YoloV3


In [8]:
dataset = LoadImagesAndLabels(train_path, img_size = 416 , batch_size = batch_size,
                                  augment=True,
                                  hyp=hyp,  # augmentation hyperparameters
                                  rect=opt.rect,  # rectangular training
                                  cache_images=opt.cache_images,
                                  single_cls=opt.single_cls)

Caching labels (6 found, 0 missing, 0 empty, 0 duplicate, for 6 images): 100%|█████████| 6/6 [00:00<00:00, 3057.82it/s]
Caching images (0.0GB): 100%|████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 36.90it/s]


In [8]:
batch_size = min(batch_size, len(dataset))
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=batch_size,
                                         num_workers=nw,
                                         shuffle=not opt.rect,  # Shuffle=True unless rectangular training is used
                                         pin_memory=True,
                                         collate_fn=dataset.collate_fn)

In [9]:
testloader = torch.utils.data.DataLoader(LoadImagesAndLabels(test_path, imgsz_test, batch_size,
                                                             hyp=hyp,
                                                             rect=True,
                                                             cache_images=opt.cache_images,
                                                             single_cls=opt.single_cls),
                                         batch_size=batch_size,
                                         num_workers=nw,
                                         pin_memory=True,
                                         collate_fn=dataset.collate_fn)

Caching labels (6 found, 0 missing, 0 empty, 0 duplicate, for 6 images): 100%|█████████| 6/6 [00:00<00:00, 1291.42it/s]
Caching images (0.0GB): 100%|████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 37.78it/s]


In [10]:
# Model parameters
model.nc = nc  # attach number of classes to model
model.hyp = hyp  # attach hyperparameters to model
model.gr = 1.0  # giou loss ratio (obj_loss = 1.0 or giou)
model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device)  # attach class weights

# Model EMA
ema = torch_utils.ModelEMA(model)
img_size = 416

In [13]:
import yolo_test
# Start training
nb = len(dataloader)  # number of batches
n_burn = max(3 * nb, 500)  # burn-in iterations, max(3 epochs, 500 iterations)
maps = np.zeros(nc)  # mAP per class
# torch.autograd.set_detect_anomaly(True)
results = (0, 0, 0, 0, 0, 0, 0)  # 'P', 'R', 'mAP', 'F1', 'val GIoU', 'val Objectness', 'val Classification'
t0 = time.time()
print('Image sizes %g - %g train, %g test' % (imgsz_min, imgsz_max, imgsz_test))
print('Using %g dataloader workers' % nw)
print('Starting training for %g epochs...' % epochs)
for epoch in range(start_epoch, epochs):# epoch ------------------------------------------------------------------
    model.train()
    mloss = torch.zeros(4).to(device)  # mean losses
    print(('\n' + '%10s' * 8) % ('Epoch', 'gpu_mem', 'GIoU', 'obj', 'cls', 'total', 'targets', 'img_size'))
    pbar = tqdm(enumerate(dataloader), total=nb)  # progress bar
    for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------
        model.train()
        
        mloss = torch.zeros(4).to(device)  # mean losses
        print(('\n' + '%10s' * 8) % ('Epoch', 'gpu_mem', 'GIoU', 'obj', 'cls', 'total', 'targets', 'img_size'))
        pbar = tqdm(enumerate(dataloader), total=nb)  # progress bar
        for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------
            ni = i + nb * epoch  # number integrated batches (since train start)
            imgs = imgs.to(device).float() / 255.0  # uint8 to float32, 0 - 255 to 0.0 - 1.0
            targets = targets.to(device)

            # Burn-in
            if ni <= n_burn * 2:
                model.gr = np.interp(ni, [0, n_burn * 2], [0.0, 1.0])  # giou loss ratio (obj_loss = 1.0 or giou)
                if ni == n_burn:  # burnin complete
                    print_model_biases(model)

                for j, x in enumerate(optimizer.param_groups):
                    # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                    x['lr'] = np.interp(ni, [0, n_burn], [0.1 if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                    if 'momentum' in x:
                        x['momentum'] = np.interp(ni, [0, n_burn], [0.9, hyp['momentum']])

            # Multi-Scale training
            if opt.multi_scale:
                if ni / accumulate % 1 == 0:  #  adjust img_size (67% - 150%) every 1 batch
                    img_size = random.randrange(grid_min, grid_max + 1) * gs
                sf = img_size / max(imgs.shape[2:])  # scale factor
                if sf != 1:
                    ns = [math.ceil(x * sf / gs) * gs for x in imgs.shape[2:]]  # new shape (stretched to 32-multiple)
                    imgs = F.interpolate(imgs, size=ns, mode='bilinear', align_corners=False)

            # Run model
            pred = model(imgs)
            yolo_pred = (pred[1],pred[2],pred[3])

            # Compute yolo loss
            loss, loss_items = compute_loss(yolo_pred, targets, model)

            # Scale loss by nominal batch_size of 64
            loss *= batch_size / 64

            # Compute gradient
            if mixed_precision:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            # Optimize accumulated gradient
            if ni % accumulate == 0:
                optimizer.step()
                optimizer.zero_grad()
                ema.update(model)

            # Print batch results
            mloss = (mloss * i + loss_items) / (i + 1)  # update mean losses
            mem = '%.3gG' % (torch.cuda.memory_cached() / 1E9 if torch.cuda.is_available() else 0)  # (GB)
            s = ('%10s' * 2 + '%10.3g' * 6) % ('%g/%g' % (epoch, epochs - 1), mem, *mloss, len(targets), img_size)
            pbar.set_description(s)
            
        # Update scheduler
        scheduler.step()

        # Process epoch results
        ema.update_attr(model)
        final_epoch = epoch + 1 == epochs
        if not opt.notest or final_epoch:  # Calculate mAP
            is_coco = any([x in data for x in ['coco.data', 'coco2014.data', 'coco2017.data']]) and model.nc == 80
            results, maps = yolo_test.test(cfg,
                                      data,
                                      batch_size=batch_size,
                                      img_size=imgsz_test,
                                      model=ema.ema,
                                      save_json=final_epoch and is_coco,
                                      single_cls=opt.single_cls,
                                      dataloader=testloader)

        # Write epoch results
        with open(results_file, 'a') as f:
            f.write(s + '%10.3g' * 7 % results + '\n')  # P, R, mAP, F1, test_losses=(GIoU, obj, cls)
        if len(opt.name) and opt.bucket:
            os.system('gsutil cp results.txt gs://%s/results/results%s.txt' % (opt.bucket, opt.name))

        # Update best mAP
        fi = fitness(np.array(results).reshape(1, -1))  # fitness_i = weighted combination of [P, R, mAP, F1]
        if fi > best_fitness:
            best_fitness = fi

        # Save training results
        save = False #(not opt.nosave) or (final_epoch and not opt.evolve)
        if save:
            with open(results_file, 'r') as f:
                # Create checkpoint
                chkpt = {'epoch': epoch,
                         'best_fitness': best_fitness,
                         'training_results': f.read(),
                         'model': ema.ema.module.state_dict() if hasattr(model, 'module') else ema.ema.state_dict(),
                         'optimizer': None if final_epoch else optimizer.state_dict()}

            # Save last checkpoint
            torch.save(chkpt, last)

            # Save best checkpoint
            if (best_fitness == fi) and not final_epoch:
                torch.save(chkpt, best)

            # Save backup every 10 epochs (optional)
            # if epoch > 0 and epoch % 10 == 0:
            #     torch.save(chkpt, wdir + 'backup%g.pt' % epoch)

            # Delete checkpoint
            del chkpt

Image sizes 512 - 512 train, 512 test
Using 6 dataloader workers
Starting training for 2 epochs...

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
       0/1     3.08G      2.34     0.443      5.23      8.01        31       416:   0%|          | 0/1 [00:00<?, ?it/s]
       0/1     3.08G      2.34     0.443      5.23      8.01        31       416: 100%|██| 1/1 [00:01<00:00,  1.01s/it]

               Class    Images   Targets         P         R   mAP@0.5        F1:   0%|          | 0/1 [00:00<?, ?it/s]
               Class    Images   Targets         P         R   mAP@0.5        F1: 100%|██| 1/1 [00:08<00:00,  8.80s/it]
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.05s/it]

                 all         6        13         0         0         0         0


...

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]
       1/1     3.88G      2.51     0.442       6.1      9.05        25       416:   0%|          | 0/1 [00:00<?, ?it/s]
       1/1     3.88G      2.51     0.442       6.1      9.05        25       416: 100%|██| 1/1 [00:01<00:00,  1.07s/it]

               Class    Images   Targets         P         R   mAP@0.5        F1:   0%|          | 0/1 [00:00<?, ?it/s]
               Class    Images   Targets         P         R   mAP@0.5        F1: 100%|██| 1/1 [00:08<00:00,  8.76s/it]
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.20s/it]

                 all         6        13         0         0         0         0


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.48s/it]


In [23]:
import test
results, maps = test.test(cfg,
                          data,
                          batch_size=batch_size,
                          img_size=imgsz_test,
                          model=ema.ema,
                          save_json=False,
                          single_cls=opt.single_cls,
                          dataloader=testloader)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not dict

In [15]:
#debug test.test 
#import json
#from torch.utils.data import DataLoader
#from model_yolo import *
#from utils_yolo.datasets import *
#from utils_yolo.utils import *

#batch_size=16,
#img_size=416,
conf_thres=torch.tensor(0.001)
iou_thres=torch.tensor(0.6)  # for nms
save_json=False
single_cls=False
augment=False

In [16]:
# Configure run
data = parse_data_cfg(opt.data)
nc = 1 if opt.single_cls else int(data['classes'])  # number of classes
path = data['valid']  # path to test images
names = load_classes(data['names'])  # class names
iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
iouv = iouv[0].view(1)  # comment for mAP@0.5:0.95
niou = iouv.numel()

In [17]:
seen = 0
model.eval()
img_size = 416
_ = model(torch.zeros((1, 3, img_size, img_size), device=device)) if device.type != 'cpu' else None
s = ('%20s' + '%10s' * 6) % ('Class', 'Images', 'Targets', 'P', 'R', 'mAP@0.5', 'F1')
p, r, f1, mp, mr, map, mf1, t0, t1 = 0., 0., 0., 0., 0., 0., 0., 0., 0.
loss = torch.zeros(3, device=device)
jdict, stats, ap, ap_class = [], [], [], []

In [18]:
verbose = False
for batch_i, (imgs, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
    imgs = imgs.to(device).float() / 255.0  # uint8 to float32, 0 - 255 to 0.0 - 1.0
    targets = targets.to(device)
    nb, _, height, width = imgs.shape  # batch size, channels, height, width
    whwh = torch.Tensor([width, height, width, height]).to(device)

    # Plot images with bounding boxes
    f = 'test_batch%g.png' % batch_i  # filename
    if batch_i < 1 and not os.path.exists(f):
        plot_images(imgs=imgs, targets=targets, paths=paths, fname=f)

    # Disable gradients
    with torch.no_grad():
        # Run model
        t = torch_utils.time_synchronized()
        model.eval()
        p = model(imgs)
        # inference
        inf_out = (p[1][0], p[2][0], p[3][0])  # inference output, training output
        inf_out = torch.cat(inf_out, 1)  # cat yolo outputs

        #training outputs
        train_out = (p[1][1], p[2][1], p[3][1])

        t0 += torch_utils.time_synchronized() - t

        # Compute loss
        if hasattr(model, 'hyp'):  # if model has loss hyperparameters
            loss += compute_loss(train_out, targets, model)[1][:3]  # GIoU, obj, cls

        # Run NMS
        t = torch_utils.time_synchronized()
        output = non_max_suppression(inf_out, conf_thres=conf_thres, iou_thres=iou_thres)  # nms
        t1 += torch_utils.time_synchronized() - t

    # Statistics per image
    for si, pred in enumerate(output):
        labels = targets[targets[:, 0] == si, 1:]
        nl = len(labels)
        tcls = labels[:, 0].tolist() if nl else []  # target class
        seen += 1

        if pred is None:
            if nl:
                stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
            continue

        # Append to text file
        # with open('test.txt', 'a') as file:
        #    [file.write('%11.5g' * 7 % tuple(x) + '\n') for x in pred]

        # Clip boxes to image bounds
        clip_coords(pred, (height, width))

        # Append to pycocotools JSON dictionary
        if save_json:
            # [{"image_id": 42, "category_id": 18, "bbox": [258.15, 41.29, 348.26, 243.78], "score": 0.236}, ...
            image_id = int(Path(paths[si]).stem.split('_')[-1])
            box = pred[:, :4].clone()  # xyxy
            scale_coords(imgs[si].shape[1:], box, shapes[si][0], shapes[si][1])  # to original shape
            box = xyxy2xywh(box)  # xywh
            box[:, :2] -= box[:, 2:] / 2  # xy center to top-left corner
            for p, b in zip(pred.tolist(), box.tolist()):
                jdict.append({'image_id': image_id,
                              'category_id': coco91class[int(p[5])],
                              'bbox': [round(x, 3) for x in b],
                              'score': round(p[4], 5)})

        # Assign all predictions as incorrect
        correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool, device=device)
        if nl:
            detected = []  # target indices
            tcls_tensor = labels[:, 0]

            # target boxes
            tbox = xywh2xyxy(labels[:, 1:5]) * whwh

            # Per target class
            for cls in torch.unique(tcls_tensor):
                ti = (cls == tcls_tensor).nonzero().view(-1)  # prediction indices
                pi = (cls == pred[:, 5]).nonzero().view(-1)  # target indices

                # Search for detections
                if pi.shape[0]:
                    # Prediction to target ious
                    ious, i = box_iou(pred[pi, :4], tbox[ti]).max(1)  # best ious, indices

                    # Append detections
                    for j in (ious > iouv[0]).nonzero():
                        d = ti[i[j]]  # detected target
                        if d not in detected:
                            detected.append(d)
                            correct[pi[j]] = ious[j] > iouv  # iou_thres is 1xn
                            if len(detected) == nl:  # all targets already located in image
                                break

        # Append statistics (correct, conf, pcls, tcls)
        stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))

# Compute statistics
stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy
if len(stats):
    p, r, ap, f1, ap_class = ap_per_class(*stats)
    if niou > 1:
        p, r, ap, f1 = p[:, 0], r[:, 0], ap.mean(1), ap[:, 0]  # [P, R, AP@0.5:0.95, AP@0.5]
    mp, mr, map, mf1 = p.mean(), r.mean(), ap.mean(), f1.mean()
    nt = np.bincount(stats[3].astype(np.int64), minlength=nc)  # number of targets per class
else:
    nt = torch.zeros(1)

# Print results
pf = '%20s' + '%10.3g' * 6  # print format
print(pf % ('all', seen, nt.sum(), mp, mr, map, mf1))

# Print results per class
if verbose and nc > 1 and len(stats):
    for i, c in enumerate(ap_class):
        print(pf % (names[c], seen, nt[c], p[i], r[i], ap[i], f1[i]))

# Print speeds
if verbose or save_json:
    t = tuple(x / seen * 1E3 for x in (t0, t1, t0 + t1)) + (img_size, img_size, batch_size)  # tuple
    print('Speed: %.1f/%.1f/%.1f ms inference/NMS/total per %gx%g image at batch-size %g' % t)


maps = np.zeros(nc) + map
for i, c in enumerate(ap_class):
    maps[c] = ap[i]

               Class    Images   Targets         P         R   mAP@0.5        F1:   0%|          | 0/1 [00:00<?, ?it/s]D:\ML\EVA\JEDI\Captsone_EVA5\utils_yolo\utils.py:880: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.
  plt.subplot(ns, ns, i + 1).imshow(imgs[i].transpose(1, 2, 0))
D:\ML\EVA\JEDI\Captsone_EVA5\utils_yolo\utils.py:880: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.
  plt.subplot(ns, ns, i + 1).imshow(imgs[i].transpose(1, 2, 0))
D:\ML\EVA\JEDI\Captsone_EVA5\utils_yolo\utils.py:880: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.
  plt.subplot(ns, ns, i + 1).imshow(imgs[i].transpose(1, 2, 0))
D:\ML\EVA\JEDI\Captsone_EVA5\utils_yolo\util

                 all         6        27         0         0         0         0


In [19]:
(mp, mr, map, mf1, *(loss.cpu() / len(dataloader)).tolist()), maps

((0.0,
  0.0,
  0.0,
  0.0,
  1.6508872509002686,
  0.1542627215385437,
  3.5399832725524902),
 array([          0,           0,           0,           0]))

In [14]:
model.eval()
imgs, targets, paths, shapes = next(iter(testloader))
imgs = imgs.to(device).float() / 255.0
targets = targets.to(device)
nb, _, height, width = imgs.shape  # batch size, channels, height, width
whwh = torch.Tensor([width, height, width, height]).to(device)

In [15]:
p = model(imgs)
p[0].size(),p[1][0].size(), p[1][1].size(), p[2][0].size(), p[2][1].size(),p[3][0].size(), p[3][1].size()

(torch.Size([6, 512, 512]),
 torch.Size([6, 768, 9]),
 torch.Size([6, 3, 16, 16, 9]),
 torch.Size([6, 3072, 9]),
 torch.Size([6, 3, 32, 32, 9]),
 torch.Size([6, 12288, 9]),
 torch.Size([6, 3, 64, 64, 9]))

In [17]:
# inference
inf_out = (p[1][0], p[2][0], p[3][0])  # inference output, training output
inf_out = torch.cat(inf_out, 1)  # cat yolo outputs

#training outputs
train_out = (p[1][1], p[2][1], p[3][1])


In [25]:
output = non_max_suppression(inf_out, conf_thres=conf_thres, iou_thres=iou_thres)
print(output)

[tensor([[1.49439e+02, 2.37761e+02, 1.62268e+02, 2.49922e+02, 1.08190e-03, 3.00000e+00]], device='cuda:0', grad_fn=<IndexBackward>), None, tensor([[2.38036e+02, 1.91530e+02, 2.52073e+02, 1.98116e+02, 1.21167e-03, 3.00000e+00]], device='cuda:0', grad_fn=<IndexBackward>), None, None, tensor([[2.46355e+02, 1.82655e+02, 2.58699e+02, 1.93825e+02, 1.24750e-03, 3.00000e+00]], device='cuda:0', grad_fn=<IndexBackward>)]


In [28]:
labels = targets[targets[:, 0] == 0, 1:]
labels
#targets

tensor([[0.00000, 0.37846, 0.24390, 0.20615, 0.13689],
        [2.00000, 0.37500, 0.41233, 0.14846, 0.10613]], device='cuda:0')

In [24]:
conf_thres=torch.tensor(0.001)
iou_thres=torch.tensor(0.6)
min_wh, max_wh = 2, 4096

for x in inf_out: 
    x = x[x[:,4] > conf_thres]
    x = x[((x[:, 2:4] > min_wh) & (x[:, 2:4] < max_wh)).all(1)]
    print(x)
    break

tensor([[1.55854e+02, 2.43841e+02, 1.28290e+01, 1.21616e+01, 1.08259e-03, 2.21529e-03, 2.89346e-03, 4.98820e-04, 9.99357e-01]], device='cuda:0', grad_fn=<IndexBackward>)


In [19]:
conf_thres=torch.tensor(0.001)
iou_thres=torch.tensor(0.6)
with torch.no_grad():
    # Run model
    t = torch_utils.time_synchronized()
    model.eval()
    p = model(imgs)
    # inference
    inf_out = (p[1][0], p[2][0], p[3][0])  # inference output, training output
    inf_out = torch.cat(inf_out, 1)  # cat yolo outputs

    #training outputs
    train_out = (p[1][1], p[2][1], p[3][1])

    t0 += torch_utils.time_synchronized() - t

    # Compute loss
    if hasattr(model, 'hyp'):  # if model has loss hyperparameters
        loss += compute_loss(train_out, targets, model)[1][:3]  # GIoU, obj, cls

    # Run NMS
    t = torch_utils.time_synchronized()
    output = non_max_suppression(inf_out, conf_thres=conf_thres, iou_thres=iou_thres)  # nms
    t1 += torch_utils.time_synchronized() - t


In [23]:
for si, pred in enumerate(output):
    labels = targets[targets[:, 0] == si, 1:]
    nl = len(labels)
    tcls = labels[:, 0].tolist() if nl else []  # target class
    seen += 1

    if pred is None:
        if nl:
            stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
        continue

    # Append to text file
    # with open('test.txt', 'a') as file:
    #    [file.write('%11.5g' * 7 % tuple(x) + '\n') for x in pred]

    # Clip boxes to image bounds
    clip_coords(pred, (height, width))

    # Append to pycocotools JSON dictionary
    if save_json:
        # [{"image_id": 42, "category_id": 18, "bbox": [258.15, 41.29, 348.26, 243.78], "score": 0.236}, ...
        image_id = int(Path(paths[si]).stem.split('_')[-1])
        box = pred[:, :4].clone()  # xyxy
        scale_coords(imgs[si].shape[1:], box, shapes[si][0], shapes[si][1])  # to original shape
        box = xyxy2xywh(box)  # xywh
        box[:, :2] -= box[:, 2:] / 2  # xy center to top-left corner
        for p, b in zip(pred.tolist(), box.tolist()):
            jdict.append({'image_id': image_id,
                          'category_id': coco91class[int(p[5])],
                          'bbox': [round(x, 3) for x in b],
                          'score': round(p[4], 5)})

    # Assign all predictions as incorrect
    correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool, device=device)
    if nl:
        detected = []  # target indices
        tcls_tensor = labels[:, 0]

        # target boxes
        tbox = xywh2xyxy(labels[:, 1:5]) * whwh

        # Per target class
        for cls in torch.unique(tcls_tensor):
            ti = (cls == tcls_tensor).nonzero().view(-1)  # prediction indices
            pi = (cls == pred[:, 5]).nonzero().view(-1)  # target indices

            # Search for detections
            if pi.shape[0]:
                # Prediction to target ious
                ious, i = box_iou(pred[pi, :4], tbox[ti]).max(1)  # best ious, indices

                # Append detections
                for j in (ious > iouv[0]).nonzero():
                    d = ti[i[j]]  # detected target
                    if d not in detected:
                        detected.append(d)
                        correct[pi[j]] = ious[j] > iouv  # iou_thres is 1xn
                        if len(detected) == nl:  # all targets already located in image
                            break

    # Append statistics (correct, conf, pcls, tcls)
    stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))


In [25]:
stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy
if len(stats):
    p, r, ap, f1, ap_class = ap_per_class(*stats)
    if niou > 1:
        p, r, ap, f1 = p[:, 0], r[:, 0], ap.mean(1), ap[:, 0]  # [P, R, AP@0.5:0.95, AP@0.5]
    mp, mr, map, mf1 = p.mean(), r.mean(), ap.mean(), f1.mean()
    nt = np.bincount(stats[3].astype(np.int64), minlength=nc)  # number of targets per class
else:
    nt = torch.zeros(1)

# Print results
pf = '%20s' + '%10.3g' * 6  # print format
print(pf % ('all', seen, nt.sum(), mp, mr, map, mf1))

                 all         7        13         0         0         0         0


In [18]:
inf_out.shape, type(conf_thres)
device = torch_utils.select_device('', batch_size=10)
device

Using CUDA device0 _CudaDeviceProperties(name='GeForce RTX 2060', total_memory=6144MB)



device(type='cuda', index=0)

In [17]:
conf_thres=0.1
for xi, x in enumerate(inf_out):
    k = x[:,4]
    type(k)
    break

In [43]:
for xi, x in enumerate(inf_out):  # image index, image inference
    # Apply conf constraint
    x = x[x[:, 4] > conf_thres]

    # Apply width-height constraint
    x = x[((x[:, 2:4] > min_wh) & (x[:, 2:4] < max_wh)).all(1)]

    # If none remain process next image
    if not x.shape[0]:
        continue

    # Compute conf
    x[..., 5:] *= x[..., 4:5]  # conf = obj_conf * cls_conf

    # Box (center x, center y, width, height) to (x1, y1, x2, y2)
    box = xywh2xyxy(x[:, :4])

    # Detections matrix nx6 (xyxy, conf, cls)
    if multi_label:
        i, j = (x[:, 5:] > conf_thres).nonzero().t()
        x = torch.cat((box[i], x[i, j + 5].unsqueeze(1), j.float().unsqueeze(1)), 1)
    else:  # best class only
        conf, j = x[:, 5:].max(1)
        x = torch.cat((box, conf.unsqueeze(1), j.float().unsqueeze(1)), 1)

    # Filter by class
    if classes:
        x = x[(j.view(-1, 1) == torch.tensor(classes, device=j.device)).any(1)]

    # Apply finite constraint
    if not torch.isfinite(x).all():
        x = x[torch.isfinite(x).all(1)]

    # If none remain process next image
    n = x.shape[0]  # number of boxes
    if not n:
        continue

    # Sort by confidence
    # if method == 'fast_batch':
    #    x = x[x[:, 4].argsort(descending=True)]

    # Batched NMS
    c = x[:, 5] * 0 if agnostic else x[:, 5]  # classes
    boxes, scores = x[:, :4].clone() + c.view(-1, 1) * max_wh, x[:, 4]  # boxes (offset by class), scores
    if method == 'merge':  # Merge NMS (boxes merged using weighted mean)
        i = torchvision.ops.boxes.nms(boxes, scores, iou_thres)
        if n < 1E4:  # update boxes as boxes(i,4) = weights(i,n) * boxes(n,4)
            # weights = (box_iou(boxes, boxes).tril_() > iou_thres) * scores.view(-1, 1)  # box weights
            # weights /= weights.sum(0)  # normalize
            # x[:, :4] = torch.mm(weights.T, x[:, :4])
            weights = (box_iou(boxes[i], boxes) > iou_thres).float() * scores[None]  # box weights
            x[i, :4] = torch.mm(weights / weights.sum(1, keepdim=True), x[:, :4]).float()  # merged boxes
    elif method == 'vision':
        i = torchvision.ops.boxes.nms(boxes, scores, iou_thres)
    elif method == 'fast':  # FastNMS from https://github.com/dbolya/yolact
        iou = box_iou(boxes, boxes).triu_(diagonal=1)  # upper triangular iou matrix
        i = iou.max(0)[0] < iou_thres

    output[xi] = x[i]

NameError: name 'min_wh' is not defined

# debug

In [30]:
x = 5
dataset[x][0].size(),dataset[x][1].size(),dataset[x][2],dataset[x][3] 

(torch.Size([3, 416, 416]),
 torch.Size([2, 6]),
 'data\\customdata\\images\\05a207d8-6378-43a2-9d1e-431ca7a944e4-unnamed - Copy.jpg',
 None)

In [14]:
imgs, targets, paths, _ = next(iter(dataloader))

In [21]:
imgs = imgs.to(device).float() / 255.0
targets = targets.to(device)
model.eval()
#pred = model(imgs)
pred = model(imgs)

In [19]:
len(pred)

4

In [22]:
pred[0].size(),pred[1][0].size(), pred[1][1].size(), pred[2][0].size(), pred[2][1].size(),pred[3][0].size(), pred[3][1].size()
#yolo_out = (pred[1][1].size(),pred[2][1].size(),pred[3][1].size())

(torch.Size([6, 416, 416]),
 torch.Size([6, 507, 9]),
 torch.Size([6, 3, 13, 13, 9]),
 torch.Size([6, 2028, 9]),
 torch.Size([6, 3, 26, 26, 9]),
 torch.Size([6, 8112, 9]),
 torch.Size([6, 3, 52, 52, 9]))

In [22]:
inf_out = (pred[1][0],pred[2][0],pred[3][0]) # inference output, training output
inf_out = torch.cat(inf_out, 1)  # cat yolo outputs
inf_out.size()

torch.Size([6, 10647, 9])

torch.Size([6, 10647, 9])

In [46]:
#x, p = zip(pred)  # inference output, training output
x = torch.cat(yolo_out, 1)



RuntimeError: Sizes of tensors must match except in dimension 2. Got 52 and 13

NameError: name 'p' is not defined

In [14]:
loss, loss_items = compute_loss(pred, targets, model)

In [15]:
#debud build_targets
nt = targets.shape[0]
tcls, tbox, indices, av = [], [], [], []
reject, use_all_anchors = True, True
gain = torch.ones(6, device=targets.device)
multi_gpu = type(model) in (nn.parallel.DataParallel, nn.parallel.DistributedDataParallel) #False

NameError: name 'targets' is not defined

In [29]:
p = pred
i = 0 
anchors = torch.from_numpy(model.yolo_anchors[0]).float().to(targets.device)
gain[2:] = torch.tensor(p[i].shape)[[3, 2, 3, 2]]
t, a = targets * gain, []
gwh = t[:, 4:6]
print(gwh.size(), anchors.size())
if nt:
    iou = wh_iou(anchors, gwh)
    if use_all_anchors:
        na = anchors.shape[0]  # number of anchors
        a = torch.arange(na).view(-1, 1).repeat(1, nt).view(-1)
        t = t.repeat(na, 1)
    else:# use best anchor only
        iou, a = iou.max(0)  # best iou and anchor

        # reject anchors below iou_thres (OPTIONAL, increases P, lowers R)
    if reject:
        j = iou.view(-1) > model.hyp['iou_t']  # iou threshold hyperparameter
        t, a = t[j], a[j]
        
b, c = t[:, :2].long().t()  # target image, class
gxy = t[:, 2:4]  # grid x, y
gwh = t[:, 4:6]  # grid w, h
gi, gj = gxy.long().t()  # grid x, y indices
indices.append((b, a, gj, gi))

gxy -= gxy.floor()  # xy
tbox.append(torch.cat((gxy, gwh), 1))  # xywh (grids)
av.append(anchors[a])  # anchor vec

# Class
tcls.append(c)
if c.shape[0]:  # if any targets
    assert c.max() < model.nc, 'Model accepts %g classes labeled from 0-%g, however you labelled a class %g. ' \
                               'See https://github.com/ultralytics/yolov3/wiki/Train-Custom-Data' % (
                                   model.nc, model.nc - 1, c.max())

torch.Size([27, 2]) torch.Size([3, 2])


In [11]:
cd D:\\ML\\EVA\\JEDI\\Captsone_EVA5

D:\ML\EVA\JEDI\Captsone_EVA5


In [12]:
img_size = 416

In [34]:
list(model.modules())

[fork(
   (encoder): enc(
     (res1): Sequential(
       (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
       (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU(inplace=True)
       (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
       (4): Sequential(
         (0): Bottleneck(
           (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (relu): ReLU(i